In [1]:
import os
import sys
import numpy as np

BOOTPATH = os.path.dirname(sys.path[0])
sys.path.insert(0, BOOTPATH)
import dpeeg
print(dpeeg.__version__)

0.2.7


In [2]:
data = dpeeg.data.BCICIV2A(subjects=[1], tmin=0, tmax=4, verbose='WARNING', 
                           transforms=dpeeg.data.transforms.ApplyFunc(
                               lambda x: np.expand_dims(x, 1)
                           ))

Tensorflow not install, you could not use those pipelines
To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


Transform dataset ...
----------------------
[ApplyFunc(func=<function <lambda> at 0x7f5bfc081750>) starting ...]
----------------
Transform done.
Loading dataset done.


In [3]:
data[1, 0]

{'train': (array([[[ 3.90625000e-07,  6.29882813e-06,  5.07812500e-06, ...,
           -4.88281250e-06, -6.39648437e-06, -2.34375000e-06],
          [-4.24804687e-06,  0.00000000e+00, -2.78320312e-06, ...,
            5.37109375e-07, -4.15039062e-06, -2.39257812e-06],
          [-4.05273438e-06,  2.14843750e-06, -2.44140625e-07, ...,
           -5.17578125e-06, -7.22656250e-06, -4.68750000e-06],
          ...,
          [-5.71289063e-06, -3.46679687e-06, -8.05664063e-06, ...,
            1.95312500e-07, -7.42187500e-06, -7.47070312e-06],
          [-5.56640625e-06, -4.29687500e-06, -1.01074219e-05, ...,
            1.70898437e-06, -5.95703125e-06, -5.61523437e-06],
          [-2.14843750e-06, -1.22070313e-06, -5.51757812e-06, ...,
            2.92968750e-06, -4.19921875e-06, -5.76171875e-06]]]),
  3)}